<a href="https://colab.research.google.com/github/B-Aarthy/CODESOFT/blob/main/image.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

import os
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, LSTM, Embedding, Dropout, add
from tensorflow.keras.utils import to_categorical

!wget -q https://github.com/jbrownlee/Datasets/releases/download/Flickr8k/Flickr8k_Dataset.zip
!wget -q https://github.com/jbrownlee/Datasets/releases/download/Flickr8k/Flickr8k_text.zip

!unzip -q Flickr8k_Dataset.zip
!unzip -q Flickr8k_text.zip

image_folder = None
for root, dirs, files in os.walk("."):
    if len(files) > 5000:
        image_folder = root
        break

print("Image folder found at:", image_folder)


def load_captions(filename):
    captions = {}
    with open(filename, 'r') as f:
        for line in f:
            tokens = line.strip().split('\t')
            image_id = tokens[0].split('.')[0]
            caption = tokens[1].lower()
            caption = ''.join([c for c in caption if c.isalpha() or c==' '])
            caption = 'startseq ' + caption + ' endseq'

            captions.setdefault(image_id, []).append(caption)
    return captions

captions = load_captions("Flickr8k.token.txt")
print("Captions loaded:", len(captions))

vgg = VGG16()
vgg = Model(inputs=vgg.inputs, outputs=vgg.layers[-2].output)

features = {}

for img_name in os.listdir(image_folder):
    if not img_name.lower().endswith(".jpg"):
        continue

    img_path = os.path.join(image_folder, img_name)

    try:
        image = load_img(img_path, target_size=(224, 224))
        image = img_to_array(image)
        image = image.reshape((1, 224, 224, 3))
        image = preprocess_input(image)

        feature = vgg.predict(image, verbose=0)
        image_id = img_name.split('.')[0]
        features[image_id] = feature

    except Exception as e:
        print("Skipping file:", img_name)

print("Image features extracted safely:", len(features))

all_captions = []
for cap_list in captions.values():
    all_captions.extend(cap_list)

tokenizer = Tokenizer()
tokenizer.fit_on_texts(all_captions)

vocab_size = len(tokenizer.word_index) + 1
max_length = max(len(c.split()) for c in all_captions)

print("Vocab size:", vocab_size)
print("Max caption length:", max_length)

def data_generator(captions, features, tokenizer, max_length, vocab_size):
    while True:
        for key, cap_list in captions.items():
            if key not in features:
                continue
            feature = features[key][0]
            for cap in cap_list:
                seq = tokenizer.texts_to_sequences([cap])[0]
                for i in range(1, len(seq)):
                    in_seq = pad_sequences([seq[:i]], maxlen=max_length)[0]
                    out_seq = to_categorical([seq[i]], num_classes=vocab_size)[0]
                    yield ([feature, in_seq], out_seq)


inputs1 = Input(shape=(4096,))
x1 = Dropout(0.5)(inputs1)
x1 = Dense(256, activation='relu')(x1)

inputs2 = Input(shape=(max_length,))
x2 = Embedding(vocab_size, 256, mask_zero=True)(inputs2)
x2 = Dropout(0.5)(x2)
x2 = LSTM(256)(x2)

x = add([x1, x2])
x = Dense(256, activation='relu')(x)
outputs = Dense(vocab_size, activation='softmax')(x)

model = Model([inputs1, inputs2], outputs)
model.compile(loss='categorical_crossentropy', optimizer='adam')

model.summary()

steps = len(captions)
generator = data_generator(captions, features, tokenizer, max_length, vocab_size)

model.fit(generator, epochs=5, steps_per_epoch=steps)

def generate_caption(model, tokenizer, feature, max_length):
    caption = 'startseq'
    for _ in range(max_length):
        seq = pad_sequences([tokenizer.texts_to_sequences([caption])[0]], maxlen=max_length)
        yhat = np.argmax(model.predict([feature, seq], verbose=0))
        word = tokenizer.index_word.get(yhat)
        if word is None:
            break
        caption += ' ' + word
        if word == 'endseq':
            break
    return caption

image_id = list(features.keys())[0]
img = Image.open(os.path.join(image_folder, image_id + ".jpg"))
plt.imshow(img)
plt.axis('off')

print("Generated Caption:")
print(generate_caption(model, tokenizer, features[image_id], max_length))




Streaming output truncated to the last 5000 lines.
Skipping file: ._2497608431_8dfefc7a1a.jpg
Skipping file: ._2729147877_c3ec3445bf.jpg
Skipping file: ._3189941492_a3f4347b1a.jpg
Skipping file: ._2920305300_a5b1b2329a.jpg
Skipping file: ._3556037801_3992ce6826.jpg
Skipping file: ._1478294229_7e1c822fea.jpg
Skipping file: ._2275372714_017c269742.jpg
Skipping file: ._3170802797_3c851bb475.jpg
Skipping file: ._3585488964_1467945775.jpg
Skipping file: ._2192333873_2a0cbe849d.jpg
Skipping file: ._661546153_9d30db6984.jpg
Skipping file: ._2260369648_e21ae6494a.jpg
Skipping file: ._3517127930_5dbddb45f6.jpg
Skipping file: ._3045613316_4e88862836.jpg
Skipping file: ._2460159430_71ab1aacfa.jpg
Skipping file: ._2695001634_127fe2f0d7.jpg
Skipping file: ._1453366750_6e8cf601bf.jpg
Skipping file: ._2947274789_a1a35b33c3.jpg
Skipping file: ._892340814_bdd61e10a4.jpg
Skipping file: ._413737417_b0a8b445e9.jpg
Skipping file: ._3262301835_9f1a49b80a.jpg
Skipping file: ._2512683710_991c9d466d.jpg
Skippi

Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_4       │ (None, 37)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_3       │ (None, 4096)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding           │ (None, 37, 256)   │  2,247,936 │ input_layer_4[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout (Dropout)   │ (None, 4096)      │          0 │ input_layer_3[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_1 (Dropout) │ (None, 37, 256)   │          0 │ embedding[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ not_equal           │ (None, 37)        │          0 │ input_layer_4[0]… │
│ (NotEqual)          │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 256)       │  1,048,832 │ dropout[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm (LSTM)         │ (None, 256)       │    525,312 │ dropout_1[0][0],  │
│                     │                   │            │ not_equal[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add (Add)           │ (None, 256)       │          0 │ dense[0][0],      │
│                     │                   │            │ lstm[0][0]        │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 256)       │     65,792 │ add[0][0]         │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 8781)      │  2,256,717 │ dense_1[0][0]     │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 6,144,589 (23.44 MB)

 Trainable params: 6,144,589 (23.44 MB)

 Non-trainable params: 0 (0.00 B)